In [1]:
import pandas as pd
import csv
import os
import seaborn as sns

from sklearn.metrics import f1_score

In [4]:
# load csv files with results from different models
results = {}
directory = '../../0_results/classification/month-models'
for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        results[os.path.splitext(filename)[0]] = pd.read_csv(os.path.join(directory, filename))
        continue
    else:
        continue

In [17]:
# calculate f1 
f1_type = 'macro'
scores = {}

for key in results:
    scores[key] = f1_score(results[key]['label'], results[key]['prediction'], average=f1_type)

score_series = {}
for model in sorted(pd.unique(pd.Series(scores.keys()).str[:18])):
    score_series[model] = []
    for key in sorted(scores):
        if model in key:
            score_series[model].append(scores[key])

out_df = pd.DataFrame.from_dict(score_series).reset_index().rename(columns={'index':'model'})
out_df.columns = out_df.columns.str.replace('bert-base-', '')
out_df['model'] = out_df['model'].apply(lambda x: f'test-0{x+1}' if x+1!=10 else f'test-{x+1}')

out_df.set_index('model', inplace=True)

#out_df['mean'] = out_df.mean(axis=1)

cm = sns.color_palette('Greens', as_cmap=True)
out_df.T.style.background_gradient(cmap=cm, axis = 0 ).format('{0:,.2%}')

model,test-01,test-02,test-03,test-04,test-05,test-06,test-07,test-08,test-09,test-10
train-01,73.92%,64.76%,58.72%,62.67%,61.85%,61.18%,70.08%,65.36%,59.08%,70.08%
train-02,72.30%,69.21%,70.65%,65.76%,66.70%,68.33%,73.41%,68.33%,64.97%,72.43%
train-03,70.15%,70.66%,64.98%,58.41%,69.09%,67.65%,65.76%,70.19%,63.44%,71.14%
train-04,74.44%,73.41%,70.65%,68.62%,67.91%,69.30%,72.76%,70.19%,59.14%,71.57%
train-05,72.21%,70.15%,69.09%,64.57%,68.87%,66.17%,73.59%,67.03%,61.53%,77.50%
train-06,74.95%,70.64%,66.17%,64.06%,67.03%,66.17%,68.62%,63.28%,61.89%,74.99%
train-07,70.64%,64.90%,58.78%,66.24%,66.07%,66.32%,74.44%,66.17%,60.52%,71.15%
train-08,73.21%,70.64%,64.60%,65.80%,67.91%,68.16%,66.47%,72.67%,61.18%,71.63%
train-09,69.09%,69.30%,66.89%,71.85%,66.70%,65.31%,71.33%,65.36%,66.70%,68.94%
train-10,77.64%,68.76%,68.05%,71.63%,69.77%,71.06%,73.71%,68.16%,67.49%,69.35%
